Let's start by setting up some simulation parameters for the market. We'll do calculations in WEI to keep close to the original code. We will convert to ETH as necessary. As a preliminary step, let's define some market constants

In [2]:
WEI = 1
GWEI = 10**9
WHOLE = 10**18

In [ ]:
PRICE_FLOOR = .001 * GWEI
SPREAD = 110 # 10 %
LIST_REWARD = .000025 * WHOLE # MKT
STAKE = .01 * WHOLE # MKT
VOTE_BY = 7 # Days. Are units right?
BACKEND_PAYMENT = .25
MAKER_PAYMENT = .25
RESERVE_PAYMENT = 1 - BACKEND_PAYMENT - RESERVE_PAYMENT
COST_PER_BYTE = 